## Fit hanger power dependence
This notebook fits hanger power dependence

In [1]:
%matplotlib qt
import qcodes as qc
import numpy as np
from characterizations import resonators as res
from qcodes.dataset.plotting import plot_by_id
from qcodes.dataset.data_set import load_by_id

import matplotlib.pyplot as plt

C:\ProgramData\Anaconda3\envs\qcodes\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


## Data init and utils


In [15]:
def get_2Ddata(dat, labx, laby, labvals):
    '''
    labx = inner sweep variable
    laby = outer sweep variable
    '''
    
    xs = np.unique(np.array(dat.get_data(labx)))
    ys = np.unique(np.array(dat.get_data(laby)))
    
    dats = (xs, ys,)
    if len(labvals)>1:
        for labval in labvals:
            data = np.array(dat.get_data(labval))
            data.shape = (len(ys), len(xs))
            dats += (data,)
    return dats

def fit_hanger_p_dep(runid):
    dat = load_by_id(runid)
    fs, Ps, S21_res, S21_ims = get_2Ddata(dat, 'frequency', 'mw_power', ['Re_S21', 'Im_S21'])
    S21s = S21_res + 1.j*S21_ims
    pars_list = []

    for kk in range(5):
        if kk == len(Ps)-1:
            p0s = res.estimate_hanger_pars(fs, S21s[kk,:])
            p0s[5] = -np.pi/4
        p0s = res.estimate_hanger_pars(fs, S21s[kk,:])
        print('P = %s: '%Ps[kk])#, p0s)
        if 0:
            plt.figure('P = %s'%Ps[kk])
            plt.clf()
            S21init = res.hanger_S21_sloped(*p0s)

            plt.plot(S21init[0].real, S21init[0].imag, 'o')
            plt.plot(S21s[kk,:].real, S21s[kk,:].imag)
            plt.plot(S21init.real, S21init.imag)

            plt.xlim(-3,3)
            plt.ylim(-3,3)
            plt.figure('%s_abs'%Ps[kk])
            plt.clf()
            plt.plot(fs, np.abs(S21s[kk,:]))
            plt.plot(fs, np.abs(S21init))
        pars, results, hanger_model, fit_report = res.fit_hanger(fs, S21s[kk,:], display_report = False)
        res.plot_results(S21s[kk,:], pars, hanger_model, fit_report, results, figlab = 'power = %.0f'%(Ps[kk]))
        pars_list += [pars]
    print('\n')
    plt.savefig('runid%s_fit_example.pdf'%runid)
    return pars_list, dat

def plot_Qs(pars_list, dat):
    fs, Ps, S21_res, S21_ims = get_2Ddata(dat, 'frequency', 'mw_power', ['Re_S21', 'Im_S21'])
    Qcs = [pars_i['Qc'].value/1e3 for pars_i in pars_list]
    Qis = [pars_i['Qi'].value/1e3 for pars_i in pars_list]
    Qls = [pars_i['Q'].value/1e3 for pars_i in pars_list]
    errQcs = [pars_i['Qc'].stderr/1e3 for pars_i in pars_list]
    errQis = [pars_i['Qi'].stderr/1e3 for pars_i in pars_list]
    errQls = [pars_i['Q'].stderr/1e3 for pars_i in pars_list]

    plt.figure('Qs_f0_%.1f'%pars_list[0]['f0'].value)
    plt.clf()
    plt.errorbar(Ps, np.array(Qcs), yerr = errQcs, fmt = 'o', fillstyle = 'none', label = r'$Q_\mathrm{c}$')
    plt.errorbar(Ps, np.array(Qis), yerr = errQis, fmt = 'o', fillstyle = 'none', label = r'$Q_\mathrm{i}$')
    plt.errorbar(Ps, np.array(Qls), yerr = errQls, fmt = 'o', fillstyle = 'none', label = r'$Q_\mathrm{loaded}$')
    plt.ylabel('Q x 1000')
    plt.xlabel('P (dBm)')
    plt.title(r'runid = %s, $f_0$ = %.1f MHz, T = %.3fK'%(runid, pars_list[0]['f0'].value, dat.get_data('temperature')[0][0]))
    plt.legend()
    plt.tight_layout()
    plt.savefig(r'runid_%s_$f_0$_%.1f_MHz.pdf'%(runid, pars_list[0]['f0'].value))
 
    print( 'Ql_low: %.1f \nQ_c_low: %.1f \nQi_low: %.1f \ndQi: %.1f'%(Qls[0], Qcs[0], Qis[0], Qis[-1]-Qis[0])) 

### loading data

In [16]:
dbpath = r"D:\Data\20180924 - Heliox\experiments.db"

qc.initialise_database()
qc.dataset.database.initialise_or_create_database_at(dbpath)
qc.dataset.database.get_DB_location()

'D:\\Data\\20180924 - Heliox\\experiments.db'

### Fitting and Plotting

In [17]:
runid = 181
pars_list, Ps = fit_hanger_p_dep(runid)
plot_Qs(pars_list, Ps)

P = -50: 
P = -45: 
P = -40: 
P = -35: 
P = -30: 


Ql_low: 77.6 
Q_c_low: 243.5 
Qi_low: 113.9 
dQi: -19.7
